# Liu & Mei (1994)'s shallow water Bingham model with CentPy in 1D

### Import packages

In [ ]:
# Install the centpy package
!pip install centpy

In [1]:
# Import numpy and centpy for the solution 
import numpy as np
import centpy

In [2]:
# Imports functions from matplotlib and setup for the animation
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

In [3]:
import os
import csv
from numpy import linalg as LA

### Equation

We solve the Ng & Mei (1994)'s shallow water power-law fluid equations in 1D (after normalization)

\begin{equation} 
\displaystyle
\partial_t 
\begin{bmatrix} h \\ q  \end{bmatrix} 
+ 
\partial_x 
\begin{bmatrix} q \\ \beta \displaystyle \frac{q^2}{h} +\displaystyle \frac{1}{2} \alpha h^2 \end{bmatrix} 
= \begin{bmatrix} 0 \\ h-(\displaystyle \frac{q}{h^2})^n \end{bmatrix}  
\end{equation}
where 
\begin{equation}
\beta \equiv \frac{2(1+2n)}{2+3n}
\end{equation}

\begin{equation}
\alpha \equiv \frac{\cos(\theta)gh_o}{u_o^2}
\end{equation}

BC: Periodic box.

Normal flow:
\begin{equation}
h=1,\;u=1
\end{equation}

In [4]:
# problem-specific params
# included in the package in an ugly way, to be fixed in a more elegant way
# but this part should also be kept for BC and initialization
n_coeff = 0.40
alpha_coeff = 1.0
# beta_coeff = 27.0
# wave_number = 0.10
# L_x = 2.0*np.pi/wave_number
L_x = 2.878
dist_amp = 0.025
beta_coeff = (2.0*(1.0+2.0*n_coeff))/(2.0+3.0*n_coeff)

In [5]:
pars = centpy.Pars1d(x_init=0.0, x_final=L_x, t_final=40.0, dt_out=1, J=250, cfl=0.25, scheme="sd3", source="rk3", n_coeff = n_coeff, strang_splitting=True)

In [6]:
# ng & mei model
class nm1d(centpy.Equation1d):
    def initial_data(self):
        x = self.x
        u = np.zeros((self.J + 4, 2))

        u[:,0] = 1.0*(1.0+dist_amp*np.sin(2.0*np.pi*x/L_x))
        u[:,1] = 1.0

        return u

    def boundary_conditions(self, u, t):
        u[0,0] = u[-4,0]
        u[0,1] = u[-4,1]

        u[1,0] = u[-3,0]
        u[1,1] = u[-3,1]

        u[-2,0] = u[2,0]
        u[-2,1] = u[2,1]

        u[-1,0] = u[3,0]
        u[-1,1] = u[3,1]

    def flux_x(self, u):
        f = np.zeros_like(u)

        f[:, 0] = u[:, 1]
        f[:, 1] = beta_coeff*(u[:, 1])**2/u[:, 0]+0.50*alpha_coeff*(u[:, 0])**2

        return f

    def spectral_radius_x(self, u):
        vel = u[:, 1]/u[:, 0]
        #vel = q/u[:, 0]
#         return 1.0 * np.abs(vel)
        return (beta_coeff*vel+np.sqrt((beta_coeff*vel)**2-beta_coeff*(vel)**2+alpha_coeff*u[:, 0]))
        
    def source(self, u, t):
        s = np.zeros_like(u)
        q = u[:, 1] 
        vel = q/u[:, 0]
        s[:, 0] = 0.0
        s[:, 1] = gravity_coeff*channel_slope*u[:,0]-fc/2.0*(vel**2)
        

### Solution

In [7]:
eqn = nm1d(pars)
soln = centpy.Solver1dNM(eqn)
soln.solve()

### Animation

In [9]:
# Animation 

# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure(figsize=(18,8))
ax1=fig.add_subplot(1,2,1)
ax2=fig.add_subplot(1,2,2)
# plt.tight_layout()
# fig.subplots_adjust(hspace=8)


# Set the labels
ax1.set_xlabel('x')
ax1.set_ylabel(r'$\tilde{h}$')
ax1.set_xlabel('x')
ax2.set_ylabel(r'$\tilde{q}$')


# Axis limits and lines
line_u=[]
for ax in [ax1, ax2]:
  ax.set_xlim(0.0, L_x)
  ax.set_ylim(0.0, 2.0)
  line_u.append(ax.plot([], [], linewidth=1, color='b', marker='o', markersize=2)[0])

plt.subplots_adjust(bottom=0.1, right=0.75, top=0.8, left = 0.07)
plt.rcParams.update({'font.family': 'Times New Roman','font.size':19})

# animation function.  This is called sequentially
j0 = slice(2,-2)
def animate(i):
    h = soln.u_n[i,j0,0]
    q = soln.u_n[i,j0,1]
    line_u[0].set_data(soln.x[j0], h)
    line_u[1].set_data(soln.x[j0], q)

plt.close()
anim = animation.FuncAnimation(fig, animate, frames=soln.Nt, interval=100, blit=False);
HTML(anim.to_html5_video())


In [51]:
np.shape(soln.u_n)

(101, 604, 3)

In [52]:
# text files output
un_shape = np.shape(soln.u_n)
frames = un_shape[0]
j0 = slice(2,-2)
for i in range(0, frames):
    t = 1.0*i
    format_string_time = f"{t:.1f}"
    file_name = 'outXYZ_%s' % format_string_time
    with open(file_name, 'w') as f:
        writer = csv.writer(f, delimiter='\t')
        writer.writerows(zip(np.transpose(soln.x[j0]),np.transpose(soln.u_n[i,j0,0])))